In [11]:
'''
File name: project.ipynb
Author: ..., Mohamed Ndoye, Raphael Strebel
Date created: 03/11/2019
Date last modified: ...
Python Version: 3.7.4
'''

# useful : https://www.sustainabilist.com/blog/chicago-data-analysis-a-internship-project

import pandas as pd
from utils import constants
from utils import clean_database

# Set auto-reload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Load the food inspections dataframe
food_inspections_DF = pd.read_csv(constants.FOOD_INSPECTIONS_PATH, sep = ',', header = 0, 
                   names = constants.FOOD_INSPECTIONS_COL_NAMES, index_col = None, error_bad_lines=False
                   )

In [30]:
food_inspections_DF.head(20)

,inspection_id,DBA_name,AKA_name,license_num,facility_type,risk,address,city,state,zip,...,result,violations,lat,lng,location,historical_wards,zip_codes,community_areas,census_tracts,wards
0,2320830,"THE HOXTON, CHICAGO","THE HOXTON, CHICAGO",2694640.0,Restaurant,Risk 2 (Medium),200 N GREEN ST,CHICAGO,IL,60607.0,...,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.885699,-87.648789,"{'latitude': '-87.64878908937915', 'longitude'...",NaN,NaN,NaN,NaN,NaN
1,2320831,OGDEN PLAZA INC.,OGDEN PLAZA INC.,2475982.0,Grocery Store,Risk 3 (Low),3459 W OGDEN AVE,CHICAGO,IL,60623.0,...,Out of Business,NaN,41.855266,-87.712402,"{'latitude': '-87.71240156240032', 'longitude'...",NaN,NaN,NaN,NaN,NaN
2,2320829,PLAZA FOOD AND LIQUOR,PLAZA FOOD AND LIQUOR,2689756.0,Grocery Store,Risk 3 (Low),3455-3459 S OGDEN AVE,CHICAGO,IL,60623.0,...,Not Ready,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2320813,PLAZA FOOD AND LIQUOR,PLAZA FOOD AND LIQUOR,2689757.0,Grocery Store,Risk 3 (Low),3455-3459 S OGDEN AVE,CHICAGO,IL,60623.0,...,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2320757,GADS HILL CENTER,GADS HILL CENTER,2698627.0,Daycare Above and Under 2 Years,Risk 1 (High),4255-4259 S ARCHER AVE,CHICAGO,IL,60632.0,...,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.816005,-87.700893,"{'latitude': '-87.70089338917239', 'longitude'...",NaN,NaN,NaN,NaN,NaN
5,2320793,TACO MARIO'S LIMITED,TACO MARIO'S LIMITED,2622418.0,Mobile Food Preparer,Risk 2 (Medium),2300 S THROOP ST,CHICAGO,IL,60608.0,...,Pass,NaN,41.850451,-87.658798,"{'latitude': '-87.65879785567869', 'longitude'...",NaN,NaN,NaN,NaN,NaN
6,2320795,TBD,THE EXCHANGE,2698572.0,Restaurant,Risk 1 (High),300 N LA SALLE ST STE 110,CHICAGO,IL,60654.0,...,Pass,NaN,41.887529,-87.632647,"{'latitude': '-87.63264683652446', 'longitude'...",NaN,NaN,NaN,NaN,NaN
7,2320768,The Manor,The Manor,22971.0,Restaurant,Risk 1 (High),4152-4158 W 47TH ST,CHICAGO,IL,60632.0,...,No Entry,NaN,41.807924,-87.728164,"{'latitude': '-87.72816394877388', 'longitude'...",NaN,NaN,NaN,NaN,NaN
8,2320762,VIREVA NURSEY SCHOOL,VIREVA NURSEY SCHOOL,2215871.0,Children's Services Facility,Risk 1 (High),1935 W 51ST ST,CHICAGO,IL,60609.0,...,Pass,51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...,41.801134,-87.673444,"{'latitude': '-87.6734440144631', 'longitude':...",NaN,NaN,NaN,NaN,NaN
9,2320719,"4884 S ARCHER INC,.","4884 S ARCHER INC,.",2678088.0,Grocery Store,Risk 3 (Low),4884 S ARCHER AVE,CHICAGO,IL,60632.0,...,Fail,NaN,41.804621,-87.719907,"{'latitude': '-87.71990706549693', 'longitude'...",NaN,NaN,NaN,NaN,NaN


In [18]:
# We can already see that the 5 last columns are always null.

df_size = len(food_inspections_DF)
print(sum(food_inspections_DF['historical_wards'].isna()) == df_size,
      sum(food_inspections_DF['zip_codes'].isna()) == df_size,
      sum(food_inspections_DF['community_areas'].isna()) == df_size,
      sum(food_inspections_DF['census_tracts'].isna()) == df_size,
      sum(food_inspections_DF['wards'].isna()) == df_size)

True True True True True


In [23]:
food_inspections_DF = clean_database.drop_all_nan_col(food_inspections_DF) 